Reference: https://github.com/dh1105/Sentence-Entailment

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install transformers
# !pip install sentencepiece
## !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

In [3]:
import pandas as pd
import re
import torch
# import torch_xla
# import torch_xla.core.xla_model as xm
from torch.utils.data import Dataset, TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pad_sequence
# from keras.preprocessing.sequence import pad_sequences
import pickle
import os
import numpy as np

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
task_a_hyp = "This is hate speech."
task_b_hyp = "This is targeted hate."
task_c_hyp = "This is aggressive hate."
# f = open('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Data/olid_train_v2.csv')
# f_a = open('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Data/bert_nli.csv', 'w')
f = open('Data/HatEval/hateval2019_en_train.csv')
f_a = open('Data/HatEval/bert_nli_train.csv', 'w')
f_a.write('tweet_id' + '\t' + 'gold_label' + '\t' + 'sentence1' + '\t' + 'sentence2' + '\n')
lines = f.readlines()
for line in lines[1:]:
  row = line.split(',')
  tweet_id = row[0].strip()
  tweet_text = ','.join(row[1:-3]).strip()
  is_hate = row[-3].strip()
  is_targeted = row[-2].strip()
  is_ag = row[-1].strip()
  if is_hate == '0':
    f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
  else:
    f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
    if is_targeted == '0':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
    else:
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
    if is_ag == '0':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp + '\n')
    else:
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp + '\n')
f.close()
f_a.close()

In [6]:
#df = pd.read_csv('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Data/bert_nli.csv', delimiter='\t')
train_df = pd.read_csv('./Data/HatEval/bert_nli_train.csv', delimiter='\t')

In [7]:
train_df = train_df.sample(frac=1)

In [8]:
train_df

,tweet_id,gold_label,sentence1,sentence2
11528,5859,entailment,@SantinaDiMaggio happy birthday ya lil freak! ...,This is aggressive hate.
4332,2587,contradiction,The biggest fear the Democrats have is Preside...,This is hate speech.
15330,8463,contradiction,HAPPY BIRTHDAY YOU SKANK @spencerMR8 https://t...,This is hate speech.
15734,8750,entailment,@kylegriffin1 Hard to figure which was the mor...,This is targeted hate.
2719,1724,contradiction,Last week UNHCR released their annual Global T...,This is hate speech.
...,...,...,...,...
16351,9084,entailment,"@suma_surendran Wooow nace pic,, is bien you p...",This is hate speech.
14562,7695,contradiction,Sometimes I just wanna crash into psychic read...,This is hate speech.
13924,7057,contradiction,RT @Hasto_McGasto: “@LACLlPPERS: #YesAllMen ht...,This is hate speech.
13231,6428,entailment,@SapphireSux Sorry the old sapphire can’t come...,This is hate speech.


In [9]:
task_a_hyp = "This is hate speech."
task_b_hyp = "This is targeted hate."
task_c_hyp = "This is aggressive hate."
# f = open('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Data/olid_train_v2.csv')
# f_a = open('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Data/bert_nli.csv', 'w')
f = open('Data/HatEval/hateval2019_en_dev.csv')
f_a = open('Data/HatEval/bert_nli_val.csv', 'w')
f_a.write('tweet_id' + '\t' + 'gold_label' + '\t' + 'sentence1' + '\t' + 'sentence2' + '\n')
lines = f.readlines()
for line in lines[1:]:
  row = line.split(',')
  tweet_id = row[0].strip()
  tweet_text = ','.join(row[1:-3]).strip()
  is_hate = row[-3].strip()
  is_targeted = row[-2].strip()
  is_ag = row[-1].strip()
  if is_hate == '0':
    f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
  else:
    f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
    if is_targeted == '0':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
    else:
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
    if is_ag == '0':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp + '\n')
    else:
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp + '\n')
f.close()
f_a.close()

In [10]:
val_df = pd.read_csv('./Data/HatEval/bert_nli_val.csv', delimiter='\t')

In [11]:
val_df = val_df.sample(frac=1)

In [12]:
val_df

,tweet_id,gold_label,sentence1,sentence2
1301,18910,entailment,@dumblefttweets @bryyelyy @tariqnasheed Stfu y...,This is aggressive hate.
327,18385,contradiction,@Jim_Jordan Americans want @SpeakerRyan to fun...,This is targeted hate.
1674,19107,entailment,"""Lol you're probably tweeting, """"it's so hard ...",This is hate speech.
1236,18875,contradiction,@JKCorden Maybe James to You? All these Women ...,This is hate speech.
1668,19103,entailment,"shut the fuck up you know youre that bitch, whore",This is hate speech.
...,...,...,...,...
572,18519,contradiction,We had plenty of diversity before the #Globali...,This is aggressive hate.
457,18455,contradiction,Legal quirk often shortens prison time for cri...,This is targeted hate.
1787,19168,contradiction,Ooh this is getting into Shakespearean levels ...,This is hate speech.
1026,18771,entailment,Bitch stfu,This is hate speech.


In [13]:
task_a_hyp = "This is hate speech."
task_b_hyp = "This is targeted hate."
task_c_hyp = "This is aggressive hate."
# f = open('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Data/olid_train_v2.csv')
# f_a = open('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Data/bert_nli.csv', 'w')
f = open('Data/HatEval/hateval2019_en_test.csv')
f_a = open('Data/HatEval/bert_nli_test.csv', 'w')
f_a.write('tweet_id' + '\t' + 'gold_label' + '\t' + 'sentence1' + '\t' + 'sentence2' + '\n')
lines = f.readlines()
for line in lines[1:]:
  row = line.split(',')
  tweet_id = row[0].strip()
  tweet_text = ','.join(row[1:-3]).strip()
  is_hate = row[-3].strip()
  is_targeted = row[-2].strip()
  is_ag = row[-1].strip()
  if is_hate == '0':
    f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
  else:
    f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
    if is_targeted == '0':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
    else:
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
    if is_ag == '0':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp + '\n')
    else:
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp + '\n')
f.close()
f_a.close()

In [14]:
test_df = pd.read_csv('./Data/HatEval/bert_nli_test.csv', delimiter='\t')

In [15]:
test_df = test_df.sample(frac=1)

In [16]:
test_df

,tweet_id,gold_label,sentence1,sentence2
1906,30767,entailment,Illegal Aliens Crossing Into America Isn't Imm...,This is aggressive hate.
927,34261,contradiction,@triana922 The fact that they are here illegal...,This is aggressive hate.
2101,33011,contradiction,@realDonaldTrump Help #BuildThatWall Visit,This is hate speech.
4410,30591,entailment,Bitch idc when Ima see you hoe!!! https://t.co...,This is hate speech.
3397,31840,contradiction,You gonna buy a whole ecosystem? is probably t...,This is hate speech.
...,...,...,...,...
284,34358,contradiction,W h o t h e H e l l are these People ? #Burnin...,This is aggressive hate.
2147,30484,contradiction,@JessicaV_CIS THIS IS FABULOUS! I HOPE THE ACL...,This is hate speech.
4737,30019,entailment,When one of your old hoes dm you saying '' oh ...,This is hate speech.
2105,31935,contradiction,@RealJamesWoods WE CAN NOT forget to vote.Plen...,This is hate speech.


In [17]:
train_df = train_df.dropna()
val_df = val_df.dropna()
test_df = test_df.dropna()

In [18]:
train_df['sentence1'] = train_df['sentence1'].astype(str)
train_df['sentence2'] = train_df['sentence2'].astype(str)

In [19]:
val_df['sentence1'] = val_df['sentence1'].astype(str)
val_df['sentence2'] = val_df['sentence2'].astype(str)

In [20]:
test_df['sentence1'] = test_df['sentence1'].astype(str)
test_df['sentence2'] = test_df['sentence2'].astype(str)

In [21]:
train_df = train_df[(train_df['sentence1'].str.split().str.len() > 0) & (train_df['sentence2'].str.split().str.len() > 0)]
val_df = val_df[(val_df['sentence1'].str.split().str.len() > 0) & (val_df['sentence2'].str.split().str.len() > 0)]
test_df = test_df[(test_df['sentence1'].str.split().str.len() > 0) & (test_df['sentence2'].str.split().str.len() > 0)]

In [22]:
train_df

,tweet_id,gold_label,sentence1,sentence2
11528,5859,entailment,@SantinaDiMaggio happy birthday ya lil freak! ...,This is aggressive hate.
4332,2587,contradiction,The biggest fear the Democrats have is Preside...,This is hate speech.
15330,8463,contradiction,HAPPY BIRTHDAY YOU SKANK @spencerMR8 https://t...,This is hate speech.
15734,8750,entailment,@kylegriffin1 Hard to figure which was the mor...,This is targeted hate.
2719,1724,contradiction,Last week UNHCR released their annual Global T...,This is hate speech.
...,...,...,...,...
16351,9084,entailment,"@suma_surendran Wooow nace pic,, is bien you p...",This is hate speech.
14562,7695,contradiction,Sometimes I just wanna crash into psychic read...,This is hate speech.
13924,7057,contradiction,RT @Hasto_McGasto: “@LACLlPPERS: #YesAllMen ht...,This is hate speech.
13231,6428,entailment,@SapphireSux Sorry the old sapphire can’t come...,This is hate speech.


In [23]:
val_df

,tweet_id,gold_label,sentence1,sentence2
1301,18910,entailment,@dumblefttweets @bryyelyy @tariqnasheed Stfu y...,This is aggressive hate.
327,18385,contradiction,@Jim_Jordan Americans want @SpeakerRyan to fun...,This is targeted hate.
1674,19107,entailment,"""Lol you're probably tweeting, """"it's so hard ...",This is hate speech.
1236,18875,contradiction,@JKCorden Maybe James to You? All these Women ...,This is hate speech.
1668,19103,entailment,"shut the fuck up you know youre that bitch, whore",This is hate speech.
...,...,...,...,...
572,18519,contradiction,We had plenty of diversity before the #Globali...,This is aggressive hate.
457,18455,contradiction,Legal quirk often shortens prison time for cri...,This is targeted hate.
1787,19168,contradiction,Ooh this is getting into Shakespearean levels ...,This is hate speech.
1026,18771,entailment,Bitch stfu,This is hate speech.


In [24]:
test_df

,tweet_id,gold_label,sentence1,sentence2
1906,30767,entailment,Illegal Aliens Crossing Into America Isn't Imm...,This is aggressive hate.
927,34261,contradiction,@triana922 The fact that they are here illegal...,This is aggressive hate.
2101,33011,contradiction,@realDonaldTrump Help #BuildThatWall Visit,This is hate speech.
4410,30591,entailment,Bitch idc when Ima see you hoe!!! https://t.co...,This is hate speech.
3397,31840,contradiction,You gonna buy a whole ecosystem? is probably t...,This is hate speech.
...,...,...,...,...
284,34358,contradiction,W h o t h e H e l l are these People ? #Burnin...,This is aggressive hate.
2147,30484,contradiction,@JessicaV_CIS THIS IS FABULOUS! I HOPE THE ACL...,This is hate speech.
4737,30019,entailment,When one of your old hoes dm you saying '' oh ...,This is hate speech.
2105,31935,contradiction,@RealJamesWoods WE CAN NOT forget to vote.Plen...,This is hate speech.


In [25]:
test_df_a = test_df[test_df.sentence2 == "This is hate speech."]
test_df_b = test_df[test_df.sentence2 == "This is targeted hate."]
test_df_c = test_df[test_df.sentence2 == "This is aggressive hate."]

In [26]:
print(len(test_df_a))
print(len(test_df_b))
print(len(test_df_c))

3000
1260
1260


In [27]:
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pickle
import os
from transformers import BertTokenizer

class MNLIDataBert(Dataset):

  def __init__(self, train_df, val_df, test_df, test_df_a, test_df_b, test_df_c):
    self.label_dict = {'entailment': 0, 'contradiction': 1}

    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df
    self.test_df_a = test_df_a
    self.test_df_b = test_df_b
    self.test_df_c = test_df_c
    self.base_path = 'mnli-data-hateval'
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    self.train_data = None
    self.val_data = None
    self.test_data = None
    self.test_data_a = None
    self.test_data_b = None
    self.test_data_c = None
    self.init_data()

  def init_data(self):
    #Saving takes too much RAM
    
    if os.path.exists(os.path.join(self.base_path, 'train_data.pkl')):
      print("Found training data")
      with open(os.path.join(self.base_path, 'train_data.pkl'), 'rb') as f:
        self.train_data = pickle.load(f)
    else:
      self.train_data = self.load_data(self.train_df)
      with open(os.path.join(self.base_path, 'train_data.pkl'), 'wb') as f:
        pickle.dump(self.train_data, f)
    if os.path.exists(os.path.join(self.base_path, 'val_data.pkl')):
      print("Found val data")
      with open(os.path.join(self.base_path, 'val_data.pkl'), 'rb') as f:
        self.val_data = pickle.load(f)
    else:
      self.val_data = self.load_data(self.val_df)
      with open(os.path.join(self.base_path, 'val_data.pkl'), 'wb') as f:
        pickle.dump(self.val_data, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data.pkl')):
      print("Found test data")
      with open(os.path.join(self.base_path, 'test_data.pkl'), 'rb') as f:
        self.test_data = pickle.load(f)
    else:
      self.test_data = self.load_data(self.test_df)
      with open(os.path.join(self.base_path, 'test_data.pkl'), 'wb') as f:
        pickle.dump(self.test_data, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_a.pkl')):
      print("Found test data a")
      with open(os.path.join(self.base_path, 'test_data_a.pkl'), 'rb') as f:
        self.test_data_a = pickle.load(f)
    else:
      self.test_data_a = self.load_data(self.test_df_a)
      with open(os.path.join(self.base_path, 'test_data_a.pkl'), 'wb') as f:
        pickle.dump(self.test_data_a, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_b.pkl')):
      print("Found test data b")
      with open(os.path.join(self.base_path, 'test_data_b.pkl'), 'rb') as f:
        self.test_data_b = pickle.load(f)
    else:
      self.test_data_b = self.load_data(self.test_df_b)
      with open(os.path.join(self.base_path, 'test_data_b.pkl'), 'wb') as f:
        pickle.dump(self.test_data_b, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_c.pkl')):
      print("Found test data c")
      with open(os.path.join(self.base_path, 'test_data_c.pkl'), 'rb') as f:
        self.test_data_c = pickle.load(f)
    else:
      self.test_data_c = self.load_data(self.test_df_c)
      with open(os.path.join(self.base_path, 'test_data_c.pkl'), 'wb') as f:
        pickle.dump(self.test_data_c, f)
    # self.train_data = self.load_data(self.train_df)
    # self.val_data = self.load_data(self.val_df)
    # self.test_data = self.load_data(self.test_df)
  def load_data(self, df):
    MAX_LEN = 512
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['sentence1'].to_list()
    hypothesis_list = df['sentence2'].to_list()
    label_list = df['gold_label'].to_list()

    for (premise, hypothesis, label) in zip(premise_list, hypothesis_list, label_list):
      premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
      hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
      pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
      premise_len = len(premise_id)
      hypothesis_len = len(hypothesis_id)

      segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

      token_ids.append(torch.tensor(pair_token_ids))
      seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(self.label_dict[label])
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
    print(len(dataset))
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    train_loader = DataLoader(
      self.train_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    val_loader = DataLoader(
      self.val_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader = DataLoader(
      self.test_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_a = DataLoader(
      self.test_data_a,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_b = DataLoader(
      self.test_data_b,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_c = DataLoader(
      self.test_data_c,
      shuffle=shuffle,
      batch_size=batch_size
    )

    return train_loader, val_loader, test_loader, test_loader_a, test_loader_b, test_loader_c

In [28]:
mnli_dataset = MNLIDataBert(train_df, val_df, test_df, test_df_a, test_df_b, test_df_c)

Found training data
Found val data
Found test data
Found test data a
Found test data b
Found test data c


In [29]:
train_loader, val_loader, test_loader, test_loader_a, test_loader_b, test_loader_c = mnli_dataset.get_data_loaders(batch_size=32)

In [30]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [31]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [32]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, correct_bias=False)

In [33]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 109,483,778 trainable parameters


In [34]:
def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

In [35]:
import time

EPOCHS = 5

def train(model, train_loader, val_loader, optimizer):  
  total_step = len(train_loader)
  #model.load_state_dict(torch.load('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Weights/nli_0.pt'))
  #model.load_state_dict(torch.load('./WeightsHateval/nli_4.pt'))
  for epoch in range(EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(train_loader):
      optimizer.zero_grad()
      print(batch_idx, len(train_loader))
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      # prediction = model(pair_token_ids, mask_ids, seg_ids)
      loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()

      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)

      loss.backward()
      optimizer.step()
      
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)
    model.eval()
    total_val_acc  = 0
    total_val_loss = 0
    with torch.no_grad():
      for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(val_loader):
        optimizer.zero_grad()
        pair_token_ids = pair_token_ids.to(device)
        mask_ids = mask_ids.to(device)
        seg_ids = seg_ids.to(device)
        labels = y.to(device)

        # prediction = model(pair_token_ids, mask_ids, seg_ids)
        loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()
        
        # loss = criterion(prediction, labels)
        acc = multi_acc(prediction, labels)

        total_val_loss += loss.item()
        total_val_acc  += acc.item()

    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    #torch.save(model.state_dict(),os.path.join('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Weights', 'nli_{}.pt'.format(epoch)))
    torch.save(model.state_dict(),os.path.join('./WeightsHateval', 'nli_{}.pt'.format(epoch)))
    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [56]:
import time
def test(model, test_loader):
  model.load_state_dict(torch.load('WeightsHateval/nli_1.pt', map_location=torch.device('cpu')))
  model.eval()
  total_test_acc  = 0
  total_test_loss = 0
  start = time.time()
  preds = []
  acts = []
  with torch.no_grad():
    #print("Prediction", "Label")
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(test_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)

      # prediction = model(pair_token_ids, mask_ids, seg_ids)
      loss, prediction = model(pair_token_ids, 
                           token_type_ids=seg_ids, 
                           attention_mask=mask_ids, 
                           labels=labels).values()
        
      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)
      preds = preds + prediction.tolist()
      acts = acts + labels.tolist()
      #print(prediction, labels)
      total_test_loss += loss.item()
      total_test_acc  += acc.item() 

  test_acc  = total_test_acc/len(test_loader)
  test_loss = total_test_loss/len(test_loader)
  end = time.time()
  hours, rem = divmod(end-start, 3600)
  minutes, seconds = divmod(rem, 60)
  print(f'test_loss: {test_loss:.4f} test_acc: {test_acc:.4f}')
  print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
  return [0 if preds[i][0] > preds[i][1] else 1 for i in range(len(preds))], acts

In [37]:
train(model, train_loader, val_loader, optimizer)

0 518
1 518
2 518
3 518
4 518
5 518
6 518
7 518
8 518
9 518
10 518
11 518
12 518
13 518
14 518
15 518
16 518
17 518
18 518
19 518
20 518
21 518
22 518
23 518
24 518
25 518
26 518
27 518
28 518
29 518
30 518
31 518
32 518
33 518
34 518
35 518
36 518
37 518
38 518
39 518
40 518
41 518
42 518
43 518
44 518
45 518
46 518
47 518
48 518
49 518
50 518
51 518
52 518
53 518
54 518
55 518
56 518
57 518
58 518
59 518
60 518
61 518
62 518
63 518
64 518
65 518
66 518
67 518
68 518
69 518
70 518
71 518
72 518
73 518
74 518
75 518
76 518
77 518
78 518
79 518
80 518
81 518
82 518
83 518
84 518
85 518
86 518
87 518
88 518
89 518
90 518
91 518
92 518
93 518
94 518
95 518
96 518
97 518
98 518
99 518
100 518
101 518
102 518
103 518
104 518
105 518
106 518
107 518
108 518
109 518
110 518
111 518
112 518
113 518
114 518
115 518
116 518
117 518
118 518
119 518
120 518
121 518
122 518
123 518
124 518
125 518
126 518
127 518
128 518
129 518
130 518
131 518
132 518
133 518
134 518
135 518
136 518
137 518
138 51

In [57]:
preds, acts = test(model, test_loader)

test_loss: 1.1076 test_acc: 0.6382
00:01:22.70


In [58]:
acc = 0
for i in range(len(preds)):
    acc += 1 if preds[i] == acts[i] else 0
print("Overall test accuracy is", acc/len(acts))

Overall test accuracy is 0.6375


In [59]:
preds_a, acts_a = test(model, test_loader_a)

test_loss: 1.6371 test_acc: 0.5151
00:00:45.00


In [60]:
ent_acts_a = 0
cont_acts_a = 0
correct_ents_a = 0
correct_conts_a = 0
for i in range(len(preds_a)):
    if acts_a[i] == 0:
        ent_acts_a += 1
        if preds_a[i] == 0:
            correct_ents_a += 1
    else:
        cont_acts_a += 1
        if preds_a[i] == 1:
            correct_conts_a += 1
            
print("Total hate examples:", ent_acts_a)
print("Correctly predicted hate examples:", correct_ents_a)
print("Accuracy for predicting hate examples:", correct_ents_a/ent_acts_a)
print("Total non hate examples:", cont_acts_a)
print("Correctly predicted non hate examples:", correct_conts_a)
print("Accuracy for predicting non hate examples:", correct_conts_a/cont_acts_a)

Total hate examples: 1260
Correctly predicted hate examples: 1204
Accuracy for predicting hate examples: 0.9555555555555556
Total non hate examples: 1740
Correctly predicted non hate examples: 340
Accuracy for predicting non hate examples: 0.19540229885057472


In [61]:
preds_b, acts_b = test(model, test_loader_b)

test_loss: 0.2630 test_acc: 0.8948
00:00:08.68


In [62]:
ent_acts_b = 0
cont_acts_b = 0
correct_ents_b = 0
correct_conts_b = 0
for i in range(len(preds_b)):
    if acts_b[i] == 0:
        ent_acts_b += 1
        if preds_b[i] == 0:
            correct_ents_b += 1
    else:
        cont_acts_b += 1
        if preds_b[i] == 1:
            correct_conts_b += 1
            
print("Total targeted hate examples:", ent_acts_b)
print("Correctly predicted targeted hate examples:", correct_ents_b)
print("Accuracy for predicting targeted hate examples:", correct_ents_b/ent_acts_b)
print("Total non targeted offensive examples:", cont_acts_b)
print("Correctly predicted non targeted hate examples:", correct_conts_b)
print("Accuracy for predicting non targeted hate examples:", correct_conts_b/cont_acts_b)

Total targeted hate examples: 529
Correctly predicted targeted hate examples: 454
Accuracy for predicting targeted hate examples: 0.8582230623818525
Total non targeted offensive examples: 731
Correctly predicted non targeted hate examples: 673
Accuracy for predicting non targeted hate examples: 0.920656634746922


In [63]:
preds_c, acts_c = test(model, test_loader_c)

test_loss: 0.6963 test_acc: 0.6729
00:00:08.68


In [64]:
ent_acts_c = 0
cont_acts_c = 0
correct_ents_c = 0
correct_conts_c = 0
for i in range(len(preds_c)):
    if acts_c[i] == 0:
        ent_acts_c += 1
        if preds_c[i] == 0:
            correct_ents_c += 1
    else:
        cont_acts_c += 1
        if preds_c[i] == 1:
            correct_conts_c += 1
            
print("Total aggressive hate examples:", ent_acts_c)
print("Correctly predicted aggressive hate examples:", correct_ents_c)
print("Accuracy for predicting aggressive hate examples:", correct_ents_c/ent_acts_c)
print("Total non aggressive examples:", cont_acts_c)
print("Correctly non aggressive examples:", correct_conts_c)
print("Accuracy for predicting non aggressive examples:", correct_conts_c/cont_acts_c)

Total aggressive hate examples: 594
Correctly predicted aggressive hate examples: 402
Accuracy for predicting aggressive hate examples: 0.6767676767676768
Total non aggressive examples: 666
Correctly non aggressive examples: 446
Accuracy for predicting non aggressive examples: 0.6696696696696697
